In [1]:
import cv2
import numpy as np
import socket
import sys

In [2]:
IP = '10.0.0.5'
PORT = 10001

In [3]:
# Create a TCP/IP socket
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Connect the socket to the port where the server is listening
server_address = (IP, PORT)
print >>sys.stderr, 'connecting to %s port %s' % server_address
sock.connect(server_address)

connecting to 10.0.0.5 port 10001


In [4]:
# params
zero = 0
infinity = 95
base = 40
alpha = 5
beta = 2
# global frame count
frame_count = 0


In [5]:
# motor states
class State(object):
    def __init__(self,_name,_offset=0):
        self.name = _name
        self.fav_fc = 0
        self.ufav_fc = 0
        self.dcycle = zero
        self.offset = _offset
    def growth(self):
        self.ufav_fc = 0
        self.fav_fc = self.fav_fc + 1
        self.dcycle = min(max(self.dcycle,base) + (alpha*self.fav_fc) + self.offset,infinity)
    def decay(self):
        self.fav_fc = 0
        self.ufav_fc = self.ufav_fc + 1
        self.dcycle = max(self.dcycle - (beta*self.ufav_fc) -self.offset,zero)
    def get_value(self):
        return self.dcycle

In [6]:
l = State("left")
r = State("right")

In [7]:
# init video capture object
cap = cv2.VideoCapture("http://10.0.0.7:8080/video?x.mjpeg")

In [8]:
# kernel for sharpen operation
kernel = np.zeros( (8,8), np.float32); kernel[4,4] = 2.0 
#Create a box filter:
boxFilter = np.ones( (8,8), np.float32) / 81.0
#Subtract the two:
kernel = kernel - boxFilter

In [9]:
datasetX = [] # [list of 6x270x480 images]
datasetY = [] # (left motor state, right motor state)
sample = []
i=0
frame_count = 0
message='q' 
while(frame_count < 1500):
    
    ret, frame = cap.read()
    
    if frame is None:
        print "frame is none; breaking"
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray,(240,135), interpolation = cv2.INTER_CUBIC)
    cv2.imshow('frame',resized)

    key_press = cv2.waitKey(1)
        
    if key_press != message and key_press != -1:
        if 'w' == chr(key_press & 255):
            l.growth(); r.growth()
        elif 's' == chr(key_press & 255):
            l.decay(); r.decay()
        elif 'a' == chr(key_press & 255):
            r.growth(); l.decay()
        elif 'd' == chr(key_press & 255):
            l.growth(); r.decay()
        else:
            message='q'
            
        # construct message of form (l,r)
        message="(%s,%s)"%(l.get_value(),r.get_value())
        print >>sys.stderr, 'sending "%s"' % message
        sock.sendall(message)
    
    if(i != 1 and i != 4):
        sample.append(np.asarray(resized).ravel())
    if(i == 5 ):
        sample.append(cv2.blur(sample[-1],(5,5)))
        sample.append(cv2.filter2D(sample[-2], -1, kernel))
        datasetX.append(sample)
        datasetY.append((l.get_value(), r.get_value()))

    i = i+1
    frame_count = frame_count + 1
    if i > 5:
        sample = []
        i = 0
    

sending "(45,45)"
sending "(55,55)"
sending "(70,53)"
sending "(90,49)"
sending "(88,43)"
sending "(84,35)"
sending "(78,25)"
sending "(70,13)"
sending "(75,45)"
sending "(73,55)"
sending "(69,70)"
sending "(63,90)"
sending "(55,88)"
sending "(45,84)"
sending "(33,78)"
sending "(45,70)"
sending "(55,60)"
sending "(70,48)"
sending "(68,34)"
sending "(64,18)"
sending "(58,0)"
sending "(50,0)"
sending "(40,0)"
sending "(45,0)"
sending "(55,0)"
sending "(70,45)"
sending "(68,43)"
sending "(64,48)"
sending "(58,58)"
sending "(50,73)"
sending "(40,93)"
sending "(28,95)"
sending "(14,93)"
sending "(0,89)"
sending "(45,94)"
sending "(55,92)"
sending "(53,88)"
sending "(49,82)"
sending "(43,74)"
sending "(35,64)"
sending "(25,52)"
sending "(13,57)"
sending "(45,67)"
sending "(43,82)"
sending "(39,95)"
sending "(33,95)"
sending "(25,95)"
sending "(15,95)"
sending "(45,93)"
sending "(55,89)"
sending "(70,83)"
sending "(90,75)"
sending "(95,65)"
sending "(95,70)"
sending "(93,68)"
sending "(89,64)

In [10]:
import pickle
pickle.dump((datasetX, datasetY), open( "../../data/1minData_halfed.pkl", "wb" ) )  

In [11]:
loaded_datasetX, loaded_datasetY = pickle.load( open( "../../data/1minData_halfed.pkl", "rb" ) )

In [ ]:
for sampl in loaded_dataset:
    cv2.imshow('loaded',sampl[-2].reshape(270,480))
    cv2.waitKey(100)

In [12]:
print len(loaded_datasetX)
print len(loaded_datasetY)
print len(loaded_datasetX[0])
print len(loaded_datasetX[1])
print len(loaded_datasetY[0])
print len(loaded_datasetY[1])
print loaded_datasetX[0][0].shape

250
250
6
6
2
2
(32400,)


In [ ]:
dataset[0][0].shape